## Ungraded Lab: Lambda Layer

This lab will show how you can define custom layers with the Lambda layer. You can either use lambda functions within the Lambda layer or define a custom function that the Lambda layer will call. Let's get started!

## Imports

In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ", device)

device:  cuda


## Prepare the Dataset

In [3]:
# Image Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

In [4]:
# Load Dataset
train_data = MNIST(root='./', train=True, download=True, transform=transform)
test_data = MNIST(root='./', train=False, download=True, transform=transform)

C:\Users\adutt\Miniconda3\envs\DL\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# DataLoader
train_loader = DataLoader(dataset=train_data,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2,
                          pin_memory=True)
val_loader = DataLoader(dataset=test_data,
                        batch_size=64,
                        shuffle=True,
                        num_workers=2,
                        pin_memory=True)

## Build the Model

In [6]:
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
        
    def forward(self, x):
        return self.lambd(x)

In [7]:
# Build the Model
modules = []
modules.append(nn.Linear(in_features=784, out_features=128))
modules.append(LambdaLayer(lambda x: torch.abs(x)))
modules.append(nn.Linear(in_features=128, out_features=10))
modules.append(nn.LogSoftmax(dim=1))

model1 = nn.Sequential(*modules)
model1.to(device)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): LambdaLayer()
  (2): Linear(in_features=128, out_features=10, bias=True)
  (3): LogSoftmax(dim=1)
)

In [8]:
# Optimizer
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [9]:
# Train the Model
EPOCHS = 5

model1.train()

for epoch in range(EPOCHS):
    running_loss = 0
    correct = 0
    
    for data in train_loader:
        images, labels = data
        images = images.view(images.shape[0], -1)
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        output = model1(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        
        running_loss += loss.item()
    
    print(f"Epoch: {epoch}, loss: {running_loss/len(train_loader)}, accuracy: {correct/len(train_loader.dataset)}")


# Evaluate Trained Model
running_loss = 0
correct = 0
    
model1.eval()
for data in val_loader:
    images, labels = data
    images = images.view(images.shape[0], -1)
    images, labels = images.to(device), labels.to(device)

    output = model1(images)
    loss = criterion(output, labels)

    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(labels.data.view_as(pred)).cpu().sum()

    running_loss += loss.item()

print(f"\nValidation - loss: {running_loss/len(val_loader)}, accuracy: {correct/len(val_loader.dataset)}")

Epoch: 0, loss: 0.3151100803452578, accuracy: 0.9065166711807251
Epoch: 1, loss: 0.14382589405287366, accuracy: 0.9563000202178955
Epoch: 2, loss: 0.11083220611852623, accuracy: 0.9659500122070312
Epoch: 3, loss: 0.0925293479949784, accuracy: 0.9713500142097473
Epoch: 4, loss: 0.07943033080608614, accuracy: 0.9746999740600586

Validation - loss: 0.10000009111953294, accuracy: 0.9692000150680542


Another way to use the Lambda layer is to pass in a function defined outside the model. The code below shows how a custom ReLU function is used as a custom layer in the model.

In [10]:
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
        
    def forward(self, x):
        return self.lambd(x)

In [11]:
# Passing a function to Lambda Layer
def my_relu(x):
    return torch.maximum(torch.FloatTensor([-0.1]).to(device), x)

# Build the Model
modules = []
modules.append(nn.Linear(in_features=784, out_features=128))
modules.append(LambdaLayer(my_relu))
modules.append(nn.Linear(in_features=128, out_features=10))
modules.append(nn.LogSoftmax(dim=1))

model2 = nn.Sequential(*modules)
model2.to(device)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): LambdaLayer()
  (2): Linear(in_features=128, out_features=10, bias=True)
  (3): LogSoftmax(dim=1)
)

In [12]:
# Optimizer
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [13]:
# Train the Model
EPOCHS = 5

model2.train()

for epoch in range(EPOCHS):
    running_loss = 0
    correct = 0
    
    for data in train_loader:
        images, labels = data
        images = images.view(images.shape[0], -1)
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        output = model2(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        
        running_loss += loss.item()
    
    print(f"Epoch: {epoch}, loss: {running_loss/len(train_loader)}, accuracy: {correct/len(train_loader.dataset)}")


# Evaluate Trained Model
running_loss = 0
correct = 0
    
model2.eval()
for data in val_loader:
    images, labels = data
    images = images.view(images.shape[0], -1)
    images, labels = images.to(device), labels.to(device)

    output = model2(images)
    loss = criterion(output, labels)

    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(labels.data.view_as(pred)).cpu().sum()

    running_loss += loss.item()

print(f"\nValidation - loss: {running_loss/len(val_loader)}, accuracy: {correct/len(val_loader.dataset)}")

Epoch: 0, loss: 0.3820210333421096, accuracy: 0.8890500068664551
Epoch: 1, loss: 0.19350487205471947, accuracy: 0.9433833360671997
Epoch: 2, loss: 0.1409561559220335, accuracy: 0.9574666619300842
Epoch: 3, loss: 0.11247728188146851, accuracy: 0.9668333530426025
Epoch: 4, loss: 0.09525075353044611, accuracy: 0.9708166718482971

Validation - loss: 0.10198885168298889, accuracy: 0.9692000150680542
